<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/BERT_on_IEMOCAP_for_ERC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical  
import string
import requests

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 4.3MB/s 


In [3]:
import tokenization

In [4]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [5]:
# def build_model(bert_layer, max_len=512):
#     input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
#     input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
#     segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

#     pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
#     clf_output = pooled_output
#     clf_output1=tf.reshape(clf_output,[1,1,1024])
#     gru_output1 = GRU(100, input_shape=(1024,), return_sequences=True)(clf_output1)
#     dp_output1 = Dropout(0.2)(gru_output1)
#     print("Reached here 2?")
#     gru_output2 = GRU(100, return_sequences=True)(dp_output1)
#     dp_output2 = Dropout(0.2)(gru_output2)
#     print("Reached here 3?")
#     gru_output3 = GRU(100)(dp_output2)
#     dp_output3 = Dropout(0.2)(gru_output3)
#     out = Dense(6, activation='softmax')(dp_output3)
    
#     model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
#     model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
#     return model

In [5]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = pooled_output
    out = Dense(100, activation='relu')(clf_output)
    out = Dense(100, activation='relu')(out)
    out = Dense(6, activation='softmax')(out)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [25]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 27.1 s, sys: 5.21 s, total: 32.3 s
Wall time: 48.1 s


In [8]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)
with open('/content/gdrive/MyDrive/iemocap/train/conversation_length.pkl', 'rb') as f:
    train_convlen = pickle.load(f)
with open('/content/gdrive/MyDrive/iemocap/test/conversation_length.pkl', 'rb') as f:
    test_convlen = pickle.load(f)

In [18]:
def arrange_sentences(data,data2):
  startlen=0
  final_sentences=[]
  for i in data:
    print(i)
    odd_sentences=[]
    for j in range(0,i):
      if j%2==0:
        final_sentences.append(data2[startlen])
        startlen=startlen+1
      else:
        odd_sentences.append(data2[startlen])
        startlen=startlen+1
    for i in odd_sentences:
      final_sentences.append(i)
  return final_sentences

In [19]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [20]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [21]:
for i in range(0,len(processed_data)):
  processed_data[i]= ' '.join(processed_data[i])
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [22]:
processed_data=arrange_sentences(train_convlen,processed_data)
test_processed_data=arrange_sentences(test_convlen,test_processed_data)

52
37
28
18
57
87
38
77
35
26
53
32
54
47
26
50
26
51
33
53
43
53
40
72
34
110
60
64
47
54
46
26
62
34
63
40
66
66
53
79
26
47
45
45
18
56
83
50
37
47
74
41
47
34
59
51
69
27
63
54
54
24
44
60
47
35
60
26
63
27
39
46
20
35
51
53
61
69
51
42
42
29
52
38
62
78
69
56
8
61
94
53
26
59
58
42
52
34
59
44
28
68
29
52
59
75
49
43
91
43
83
23
84
44
54
83
28
58
58
64
43
42
53
59
44
37
40


In [32]:
# def add_ConceptNet_Features(data):
#   print(len(data))
#   for sentences in range(0,len(data)):
#     print(sentences)
#     conceptnet_data=[]
#     cleaned_sentences=data[sentences].translate(str.maketrans('', '', string.punctuation))
#     tokens_with_sw=nltk.word_tokenize(cleaned_sentences)
#     tokens= [word for word in tokens_with_sw if not word in stopwords.words()]
#     for i in tokens:
#       obj = requests.get('http://api.conceptnet.io/related/c/en/' + i + '?filter=/c/en').json()
#       for j in range(1,3):
#         response=obj['related'][j]['@id']
#         response=re.sub(r'[^\w]', ' ', response)
#         response=response[6:]
#         conceptnet_data.append(response)
#     for i in conceptnet_data:
#       data[sentences]=data[sentences]+" "+str(i)
#   return data

In [ ]:
# processed_data=add_ConceptNet_Features(processed_data)
# test_processed_data = add_ConceptNet_Features(test_processed_data)

**End of Preprocessing Here. Further Processing is for BERT**

In [23]:
processed_data=np.asarray(processed_data)
test_processed_data=np.asarray(test_processed_data)
Y=to_categorical(processed_label, num_classes=6)
test_Y=to_categorical(test_processed_label, num_classes=6)

In [26]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [27]:
train_input = bert_encode(processed_data, tokenizer, max_len=160)
test_input = bert_encode(test_processed_data, tokenizer, max_len=160)
train_labels = Y
test_labels = test_Y

In [ ]:
model = build_model(bert_layer, max_len=160)
model.summary()

In [ ]:
train_history = model.fit(
    train_input, train_labels,
    epochs=6,
    batch_size=1
)

model.save('model.h5')

In [ ]:
y_pred=model.predict(test_input, batch_size=1)

In [ ]:
y_pred1=np.argmax(y_pred,axis=1)
test_processed_label=np.asarray(test_processed_label)
print("Accuracy : " + str(accuracy_score(test_processed_label, y_pred1)))
print("Weighted F1-score : " + str(f1_score(test_processed_label, y_pred1, average='weighted')))